In [1]:
import pandas as pd
import os

In [18]:
file_path = "../data/raw/raw_sensor_data.csv"
df = pd.read_csv(file_path)
df.head(2)

,Timestamp,Machine_ID,Sensor_ID,Reading
0,2023-01-01 00:00:00,Machine_1,Sensor_1,109.934283
1,2023-01-01 00:00:00,Machine_1,Sensor_2,97.234714


In [10]:
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [19]:
scaler = preprocessing.MinMaxScaler()
scaled_reading = scaler.fit_transform(df['Reading'].values.reshape(-1, 1))
df['Reading'] = scaled_reading
df.head(2)

,Timestamp,Machine_ID,Sensor_ID,Reading
0,2023-01-01 00:00:00,Machine_1,Sensor_1,0.521518
1,2023-01-01 00:00:00,Machine_1,Sensor_2,0.432927


In [20]:
# Handle missing values (if any)
df = df.dropna()

In [21]:
# Dropping Sensor
df = df.drop(columns=['Machine_ID'])
df.head(2)

,Timestamp,Sensor_ID,Reading
0,2023-01-01 00:00:00,Sensor_1,0.521518
1,2023-01-01 00:00:00,Sensor_2,0.432927


In [23]:
# Perform feature engineering (if applicable)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['hour_of_day'] = df['Timestamp'].dt.hour
df['day_of_week'] = df['Timestamp'].dt.dayofweek
df['year'] = df['Timestamp'].dt.year
df.head(2)

,Timestamp,Sensor_ID,Reading,hour_of_day,day_of_week,year
0,2023-01-01,Sensor_1,0.521518,0,6,2023
1,2023-01-01,Sensor_2,0.432927,0,6,2023


In [25]:
df = df.drop('Timestamp',axis=1)

In [26]:
df.head(2)

,Sensor_ID,Reading,hour_of_day,day_of_week,year
0,Sensor_1,0.521518,0,6,2023
1,Sensor_2,0.432927,0,6,2023


In [27]:
from sklearn.model_selection import train_test_split
import yaml


In [28]:
split=yaml.safe_load(open('params.yaml'))['prepare']['split']
print(split)

0.2


In [32]:
features = df.drop('Reading', axis=1)
label = df['Reading']
features = pd.get_dummies(features)
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=split,random_state=42)


In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Train a Random Forest regressor
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [34]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.02186430296742853


In [35]:
# Create train and test DataFrames
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [36]:
train_df.to_csv('../data/prepared/train/train.csv')
test_df.to_csv('../data/prepared/test/test.csv')